In [26]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as mlt
import seaborn as sp
from torch.autograd import Variable
from torch import autograd
from datetime import datetime
import matplotlib.pyplot as plt
import argparse
from datetime import timedelta

In [9]:
#raw_data_csv = "Data/Raw/boulder_2021.csv"
resample_frequency = 1
split_date = '2021-12-31 23:00:00'
processed_data_csv = "Data/Processed/palo_alto_data_with_zero.csv"
missing_ratio = 0.40
lag_size = 7
train_ratio = 0.70
learning_rate = 0.001
epoch = 1000
hidden_size = 28
input_size = 7
batch_size = 256
train_dis = 5
clip = 0.01
lambda_term = 10
z_input_size = 6
future_step = 1

In [10]:
def parse_args():
    parser = argparse.ArgumentParser(description="Prepare data for imputation model training and evaluation.")
    parser.add_argument("--raw_data_csv",   type=str,   default="../Data/Raw/boulder_2021.csv", help="Path to raw data root")
        
    parser.add_argument("--day_size",             type=int,   default=48, help="Size of a day")
    parser.add_argument("--n_days",               type=int,   default=5, help="Number of days")
    parser.add_argument("--day_stride",           type=int,   default=1, help="Day stride for sliding window")
    parser.add_argument("--resample_frequency",   type=int,   default=1, help="Resample dataset in 1 hour interval")

    args = parser.parse_args()
    return args

In [11]:
def pre_process_dataset(raw_data_csv, dataset_type):
    def build_dataset(df):
        df['Start'] =  pd.to_datetime(df['Start'])
        df['End'] =  pd.to_datetime(df['End'])
        return df

    def resample_dataset(df, frequency):
        hourly_data = []
        for _, row in df.iterrows():
            plugin_time = row['Start']
            plugout_time = row['End']
            total_energy = row['Energy']
            if 'Charge.Duration' in df.columns:
                total_charging_duration = row['Charge.Duration']
            else:
                total_charging_duration = row['Park.Duration']
            #total_charging_duration = row['Charge.Duration']
            enery_per_minute = total_energy / total_charging_duration
            # Generate hourly rows
            while plugin_time < plugout_time and total_charging_duration > 0:
                # Round down to the nearest hour
                start_time = plugin_time.replace(minute=0, second=0, microsecond=0)

                # Add one hour
                next_start_time = start_time + timedelta(hours=1)

                if next_start_time > plugout_time:
                    break
                    
                time_diff = (next_start_time - plugin_time).total_seconds() / 60
                time_diff = min(time_diff, total_charging_duration)
                enery_consumption = time_diff * enery_per_minute
                total_charging_duration = total_charging_duration - time_diff

                hourly_data.append({
                    'Start': start_time,
                    'End': next_start_time,
                    'Day': row['Day'],
                    'Energy': enery_consumption,
                    'Time Duration': time_diff,
                })
                
                plugin_time = next_start_time
            if total_charging_duration == 0:
                continue
            enery_consumption = total_charging_duration * enery_per_minute
            hourly_data.append({
                'Start': start_time,
                'End': next_start_time,
                'Day': row['Day'],
                'Energy': enery_consumption,
                'Time Duration': plugout_time.minute,
            })

        # Create a new DataFrame from the hourly data
        hourly_df = pd.DataFrame(hourly_data)
        return hourly_df
    
    def aggregate_dataset(df, frequency):
        df = df.groupby('Start').agg({
            'End': 'first',
            'Day': 'first',
            'Energy': 'sum',
        }).reset_index()

        df.set_index('Start', inplace=True)
        df = df.resample('1H').asfreq()
        df.reset_index(inplace=True)
        df.loc[:,'Day'] = df['Start'].dt.dayofweek + 1
        df = df.astype({'Day': 'int32'})
        df['Week Day'] = (df['Day'] <= 5).astype(int)
        df.rename(columns={'Day': 'Day of week'}, inplace=True)
        df['Year'] = df['Start'].dt.year
        df['Month'] = df['Start'].dt.month
        df['Day of month'] = df['Start'].dt.day
        df.drop(columns=['End'], inplace=True)
        
        return df

    def interpolate_data(df, type=None):
        if type == None:
            df['Energy'] = df['Energy'].replace(np.nan, 0)
        elif type == 'linear':
            df['Energy'] = df['Energy'].interpolate(method = type, order = 2)
        else:
            df['Energy'] = df['Energy'].replace(np.nan, 0)
            df['Energy'] = df.groupby(df['Start'].dt.date)['Energy'].cumsum()
        return df

    def create_different_dataset(df, dataset_type):
        df_with_zero = interpolate_data(df.copy())
        df_with_zero.to_csv('Data/Processed/' + dataset_type + '_data_with_zero.csv', index=False)

        # df_with_imputation = interpolate_data(df.copy(), 'linear');
        # df_with_imputation.to_csv('Data/Processed/' + dataset_type + '_data_with_imputation.csv', index=False)

        # df_with_cumsum = interpolate_data(df.copy(), 'sum')
        # df_with_cumsum.to_csv('Data/Processed/' + dataset_type + '_data_with_cumsum.csv', index=False)

    df = pd.read_csv(raw_data_csv)
    
    df = remove_outliers(df)
    df = df.loc[df['Start'] <= split_date]

    df = build_dataset(df)
    df = resample_dataset(df, resample_frequency)
    df = aggregate_dataset(df, resample_frequency)

    create_different_dataset(df, dataset_type)

In [12]:
def remove_outliers(df):
    column_name = 'Energy'
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
        
    # Define the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
        
    # Filter the DataFrame to keep only the non-outliers
    df_no_outliers = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
    return df_no_outliers

In [13]:
def random_index_noise(x, ratio):
    # Get random row indices to set as null
    random_row_indices = np.random.choice(x.index, size=ratio, replace=False)

    # Set the values in the first column of the randomly selected rows to null
    x.loc[random_row_indices, 'Energy'] = np.nan
    return x

In [14]:
def prepare_dataset(dataset):
    missing_size = round(missing_ratio * len(dataset))
    real_dataset = dataset # Real dataset with no missing data
    missing_data = random_index_noise(real_dataset.copy(), missing_size) # Real dataset with missing data
    mask = np.isnan(missing_data) # Mask for tracking the indexes of the null values

    # Convert to nd array
    real = np.array(real_dataset)
    missing = np.array(missing_data.replace(np.nan, 0))
    mask = np.array(mask.replace([False, True],  [0, 1]))
    return missing, mask, real

In [15]:
def filter_data_by_lag_size(dataset, lag):
    X_purified = []
    for i in range(len(dataset)):
        if (len(dataset[i]) == lag):
            X_purified.append(dataset[i])
    return X_purified

In [16]:
def input_transform(real, lag, future_step = 0):
    X_real = []
    for i in range(len(real) - lag - future_step):
        lag_data = []
        for j in range(i, i+lag):
            lag_data.append(real[j])
        X_real.append(lag_data)
            
    X_real = np.stack(filter_data_by_lag_size(X_real, lag))

    return X_real

In [17]:
def train_test_split(X, train_ratio):
    X_train = X[0: int(len(X) * train_ratio)]
    X_test = X[int(len(X) * train_ratio): len(X)]

    return X_train, X_test

In [18]:
def get_train_test_dataset_imputation(df, train_test_ratio):
    missing, mask, real = prepare_dataset(df)
    missing = input_transform(missing, lag_size)
    mask = input_transform(mask, lag_size)
    real = input_transform(real, lag_size)
    missing_train, missing_test = train_test_split(missing, train_test_ratio)
    real_train, real_test = train_test_split(real, train_test_ratio)
    mask_train, mask_test = train_test_split(mask, train_test_ratio)

    return missing_train, real_train, mask_train, missing_test, real_test, mask_test

In [25]:
def get_forecasting_ground_truth_data(load, window, num):
    train_label = []

    for i in range(0, len(load) - window - num, num):
        for j in range(i + window, i + window + num):
            train_label.append(load[j])

    train_label = np.stack(train_label)
    return train_label

In [26]:
def get_train_test_dataset_forecasting(df, train_test_ratio):
    missing, mask, real = prepare_dataset(df)
    data = input_transform(real, lag_size, future_step)
    ground_truth = get_forecasting_ground_truth_data(real, lag_size, future_step)
    data_train, data_test = train_test_split(data, train_test_ratio)
    ground_truth_train, ground_truth_test = train_test_split(ground_truth, train_test_ratio)

    return data_train, data_test, ground_truth_train, ground_truth_test